# Download th Pdf and xmls files

## Get the annotations

In [3]:
!ls ../../edspdf-train/data/publaynet/

dev.json			test	       train	       xmls
oa_file_list.ARTgjjvC.csv.part	test_urls.txt  train_urls.txt  xmls-bis


In [2]:
from pathlib import Path
import json

with open(Path("../../edspdf-train/data/publaynet/dev.json")) as f:
  samples = json.load(f)

print(f' nb pdfs {len(samples["annotations"])}')
print(f'schema {samples.keys()}')
print(f'Schema annotations:\n\tTaille {len(samples["annotations"])} -->  {samples["annotations"][0].keys()}')
print(f'Schema images:\n\tTaille {len(samples["images"])} -->  {samples["images"][0].keys()}')
print(f'Schema categories:\n\tTaille {len(samples["categories"])} -->  {samples["categories"][0].keys()}\n\t{samples["categories"]}')

 nb pdfs 120761
schema dict_keys(['images', 'annotations', 'categories'])
Schema annotations:
	Taille 120761 -->  dict_keys(['segmentation', 'area', 'iscrowd', 'image_id', 'bbox', 'category_id', 'id'])
Schema images:
	Taille 11245 -->  dict_keys(['file_name', 'width', 'id', 'height'])
Schema categories:
	Taille 5 -->  dict_keys(['supercategory', 'id', 'name'])
	[{'supercategory': '', 'id': 1, 'name': 'text'}, {'supercategory': '', 'id': 2, 'name': 'title'}, {'supercategory': '', 'id': 3, 'name': 'list'}, {'supercategory': '', 'id': 4, 'name': 'table'}, {'supercategory': '', 'id': 5, 'name': 'figure'}]


## Dowloading the xml files with their pdfs

In [4]:
annotation_ids = {
  sample_img['file_name'].split('_')[0]
  for sample_img in samples['images']
}
len(annotation_ids)

7797

In [4]:
list(annotation_ids)[:10]

['PMC2687932',
 'PMC3747415',
 'PMC4197285',
 'PMC4678637',
 'PMC5537629',
 'PMC4213476',
 'PMC3196213',
 'PMC3375141',
 'PMC3033126',
 'PMC5723954']

In [ ]:
!head ../../edspdf-train/data/publaynet/oa_file_list.ARTgjjvC.csv.part

In [6]:
from tqdm import tqdm
import requests
import tarfile

NB_PDFs = 1000
ptrain = 0.8
ntrain = int(NB_PDFs * ptrain)

oa_package = 'https://ftp.ncbi.nlm.nih.gov/pub/pmc/'
train_dir = '../../edspdf-train/data/publaynet/train/'
test_dir = '../../edspdf-train/data/publaynet/test/'

train_urls, test_urls = [], []


with open("../../edspdf-train/data/publaynet/oa_file_list.ARTgjjvC.csv.part") as f:
    i = 0
    for line_idx, line in tqdm(enumerate(f)):
        line = line.strip()
        fname = line.split(",")[0]
        pmc_id = fname.split("/")[-1].split(".")[0]
        url = oa_package + fname
        fname = fname.split('/')[-1]

        if pmc_id in annotation_ids:
            i += 1
            if i <= ntrain:
                dir = train_dir
                train_urls.append(url)
            else:
                dir = test_dir
                test_urls.append(url)
            # print(i,'/', NB_PDFs,"PMC ID", pmc_id, "=>", url, "fname", fname, end='\r')

            # telechazrger le tar.gz
            r = requests.get(url)
            open(dir+fname, 'wb').write(r.content)

            # decompresser dans un dossier
            tar = tarfile.open(dir + fname)
            tar.extractall(dir)
            tar.close()

        if i >= NB_PDFs:
            break

# saving train_urls and test_urls
with open('../../edspdf-train/data/publaynet/train_urls.txt', 'w') as f:
    f.write('\n'.join(train_urls))
with open('../../edspdf-train/data/publaynet/test_urls.txt', 'w') as f:
    f.write('\n'.join(test_urls))

print(f'Dowloaded {len(train_urls)} train examples on dir: {train_dir}')
print(f'Dowloaded {len(test_urls)} test examples on dir: {test_dir}')
print(f'Dowloaded {NB_PDFs} train+test examples.')

374996it [32:32, 192.01it/s]  

Dowloaded 800 train examples on dir: ../../edspdf-train/data/publaynet/train/
Dowloaded 200 test examples on dir: ../../edspdf-train/data/publaynet/test/
Dowloaded 1000 train+test examples.


In [7]:
# !rm ../../edspdf-train/data/publaynet/train/*.tar.gz

rm: cannot remove '../../edspdf-train/data/publaynet/train/*.tar.gz': No such file or directory


In [8]:
# !rm ../../edspdf-train/data/publaynet/test/*.tar.gz

## Check if all of them have a pdf with a xml

In [101]:
import os
import numpy as np


xmls_names = {} # {pmc = {xml_fname, pdf_fname}}

for root, directories, files in os.walk(dir):
    for pmc in directories:
        fdir = dir + pmc
        for root2, directories2, files2 in os.walk(fdir):
            xml_fname = None
            pdfs_fnames = []
            for file in files2:
                if file.split('.')[-1] == 'nxml':
                    xml_fname = file
                if file.split('.')[-1] == 'pdf':
                    pdfs_fnames.append(file)

            lenghts = [len(pf) for pf in pdfs_fnames]
            if len(lenghts) == 0:
                print(f'fdir ==> {fdir}')

            i_min = np.argmin(les_lens)
            xmls_names[pmc] = {'xml' :xml_fname, 'pdf': pdf_fnames[i_min]}
        break

In [102]:
xmls_names

{'PMC1253713': {'xml': 'ehp0113-000073.nxml',
  'pdf': 'glasgowmedj75691-0063.pdf'},
 'PMC1247573': {'xml': 'ehp0112-001428.nxml',
  'pdf': 'glasgowmedj75691-0063.pdf'},
 'PMC1247188': {'xml': 'ehp0112-000959.nxml',
  'pdf': 'glasgowmedj75691-0063.pdf'},
 'PMC1281266': {'xml': 'ehp0113-001277.nxml',
  'pdf': 'glasgowmedj75691-0063.pdf'},
 'PMC1281292': {'xml': 'ehp0113-001430.nxml',
  'pdf': 'glasgowmedj75691-0063.pdf'},
 'PMC1280340': {'xml': 'ehp0113-001001.nxml',
  'pdf': 'glasgowmedj75691-0063.pdf'},
 'PMC1247186': {'xml': 'ehp0112-000944.nxml',
  'pdf': 'glasgowmedj75691-0063.pdf'},
 'PMC1332659': {'xml': 'ehp0114-000070.nxml',
  'pdf': 'glasgowmedj75691-0063.pdf'},
 'PMC1277117': {'xml': 'ehp0112-001236.nxml',
  'pdf': 'glasgowmedj75691-0063.pdf'},
 'PMC1247517': {'xml': 'ehp0112-001282.nxml',
  'pdf': 'glasgowmedj75691-0063.pdf'},
 'PMC1310933': {'xml': 'ehp0113-001651.nxml',
  'pdf': 'glasgowmedj75691-0063.pdf'},
 'PMC1281267': {'xml': 'ehp0113-001285.nxml',
  'pdf': 'glasgowme